In [3]:
import pandas as pd
import numpy as np
import os

### Read the UN Comtrade data

In [20]:

# Specify the path to the folder containing CSV files
folder_path = '../data/raw/MonthlyTradeData_HS8542'

# Get a list of all files in the folder
files = [file for file in os.listdir(folder_path)]

# Create an empty list to store DataFrames
dfs = []

# Loop through each CSV file and read it into a DataFrame
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, encoding='unicode_escape')
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
monthly_trade_data = pd.concat(dfs)

# Keep the necessary columns
monthly_trade_data = monthly_trade_data[
    ['Period', 'ReporterISO', 'ReporterDesc', 'PartnerISO', 'PartnerDesc', 'PrimaryValue']
]

# Convert the 'Period' column to datetime format
monthly_trade_data['Period'] = pd.to_datetime(monthly_trade_data['Period'], format='%Y%m')

# Filter the 'Period' to keep data between 2010-01-01 and 2023-08-01
monthly_trade_data = monthly_trade_data[
    (monthly_trade_data['Period'] >= '2010-01-01') & 
    (monthly_trade_data['Period'] <= '2023-08-01')
]


In [21]:

# Specify the path to the folder containing CSV files
folder_path_tw = '../data/raw/8542_extra'

# Get a list of all files in the folder
files = os.listdir(folder_path_tw)

# Create an empty list to store DataFrames
dfs = []

# Loop through each CSV file and read it into a DataFrame
for file in files:
    file_path = os.path.join(folder_path_tw, file)
    df = pd.read_csv(file_path, encoding='unicode_escape')
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
monthly_trade_data_tw = pd.concat(dfs)

# Keep the necessary columns
monthly_trade_data_tw = monthly_trade_data_tw[
    ['Period', 'ReporterISO', 'ReporterDesc', 'PartnerISO', 'PartnerDesc', 'PrimaryValue']
]

# Convert the 'Period' column to datetime format
monthly_trade_data_tw['Period'] = pd.to_datetime(monthly_trade_data_tw['Period'], format='%Y%m')

# Filter the 'Period' to keep data between 2010-01-01 and 2023-08-01
monthly_trade_data_tw = monthly_trade_data_tw[
    (monthly_trade_data_tw['Period'] >= '2010-01-01') & 
    (monthly_trade_data_tw['Period'] <= '2023-08-01')
]

# Exchange the positions of Reporter and Partner since we want to use the import data as an addition to export data of TW
monthly_trade_data_tw.rename(columns={
    'ReporterISO': 'PartnerISO_1',
    'ReporterDesc': 'PartnerDesc_1',
    'PartnerISO': 'ReporterISO_1',
    'PartnerDesc': 'ReporterDesc_1'
}, inplace=True)

monthly_trade_data_tw.rename(columns={
    'ReporterISO_1': 'ReporterISO',
    'ReporterDesc_1': 'ReporterDesc',
    'PartnerISO_1': 'PartnerISO',
    'PartnerDesc_1': 'PartnerDesc'
}, inplace=True)

# Extract the needed columns
monthly_trade_data_tw = monthly_trade_data_tw[
    ['Period', 'ReporterISO', 'ReporterDesc', 'PartnerISO', 'PartnerDesc', 'PrimaryValue']
]


In [22]:

# Concatenate the two DataFrames into a single DataFrame
monthly_trade_data = pd.concat([monthly_trade_data, monthly_trade_data_tw])

# Display the first few rows of the concatenated DataFrame
print(monthly_trade_data.head())

# Display the shape of the concatenated DataFrame
print(monthly_trade_data.shape)


      Period ReporterISO ReporterDesc PartnerISO PartnerDesc  PrimaryValue
0 2015-01-01         DZA      Algeria        W00       World      2628.566
1 2015-01-01         DZA      Algeria        USA         USA      2628.566
2 2015-01-01         AGO       Angola        W00       World      2982.590
3 2015-01-01         AGO       Angola        FRA      France       495.050
4 2015-01-01         AGO       Angola        NAM     Namibia      2426.630
(506633, 6)


In [23]:

# Function to strip leading and trailing spaces from string values
def strip_categorical(value):
    if isinstance(value, str):
        return value.strip()
    return value

# Function to replace country codes and clean DataFrame
def replace_and_delete_countries(df):
    # Apply the strip_categorical function to the entire DataFrame
    df = df.applymap(strip_categorical)
    
    # Replace "Other Asia, nes" (S19) with Taiwan, China (TWN)
    df['ReporterISO'] = df['ReporterISO'].replace('S19', 'TWN')
    df.loc[df['ReporterISO'] == 'TWN', 'ReporterDesc'] = 'Taiwan'
    
    df['PartnerISO'] = df['PartnerISO'].replace('S19', 'TWN')
    df.loc[df['PartnerISO'] == 'TWN', 'PartnerDesc'] = 'Taiwan'
    
    # Drop rows containing NaN
    df = df.dropna()
    
    # List of 193 ISO codes for UN members plus Taiwan
    to_keep = [
        "AFG", "ALB", "DZA", "AND", "AGO", "ATG", "ARG", "ARM", "AUS", "AUT", "AZE", "BHS", 
        "BHR", "BGD", "BRB", "BLR", "BEL", "BLZ", "BEN", "BTN", "BOL", "BIH", "BWA", "BRA", 
        "BRN", "BGR", "BFA", "BDI", "CPV", "KHM", "CMR", "CAN", "CAF", "TCD", "CHL", "CHN", 
        "COL", "COM", "COG", "COD", "CRI", "CIV", "HRV", "CUB", "CYP", "CZE", "DNK", "DJI", 
        "DMA", "DOM", "ECU", "EGY", "SLV", "GNQ", "ERI", "EST", "SWZ", "ETH", "FJI", "FIN", 
        "FRA", "GAB", "GMB", "GEO", "DEU", "GHA", "GRC", "GRD", "GTM", "GIN", "GNB", "GUY", 
        "HTI", "HND", "HUN", "ISL", "IND", "IDN", "IRN", "IRQ", "IRL", "ISR", "ITA", "JAM", 
        "JPN", "JOR", "KAZ", "KEN", "KIR", "KWT", "KGZ", "LAO", "LVA", "LBN", "LSO", "LBR", 
        "LBY", "LIE", "LTU", "LUX", "MDG", "MWI", "MYS", "MDV", "MLI", "MLT", "MHL", "MRT", 
        "MUS", "MEX", "FSM", "MDA", "MCO", "MNG", "MNE", "MAR", "MOZ", "MMR", "NAM", "NRU", 
        "NPL", "NLD", "NZL", "NIC", "NER", "NGA", "PRK", "MKD", "NOR", "OMN", "PAK", "PLW", 
        "PAN", "PNG", "PRY", "PER", "PHL", "POL", "PRT", "QAT", "ROU", "RUS", "RWA", 
        "KNA", "LCA", "VCT", "WSM", "SMR", "STP", "SAU", "SEN", "SRB", "SYC", "SLE", "SGP", 
        "SVK", "SVN", "SLB", "SOM", "ZAF", "KOR", "SSD", "ESP", "LKA", "SDN", "SUR", "SWE", 
        "CHE", "SYR", "TJK", "TZA", "THA", "TLS", "TGO", "TON", "TTO", "TUN", "TUR", "TKM", 
        "TUV", "UGA", "UKR", "ARE", "GBR", "USA", "URY", "UZB", "VUT", "VEN", "VNM", "YEM", 
        "ZMB", "ZWE", "TWN"
    ]
    
    # Keep rows where ReporterISO and PartnerISO are in the list of countries to keep
    df = df[df['ReporterISO'].isin(to_keep) & df['PartnerISO'].isin(to_keep)]
    
    return df

# Apply the function on monthly_TradeData
monthly_trade_data = replace_and_delete_countries(monthly_trade_data) 

# Print the shape of the DataFrame after processing
print("Processed shape:", monthly_trade_data.shape)

# Display a concise summary of the DataFrame
monthly_trade_data.info()


Processed shape: (460996, 6)
<class 'pandas.core.frame.DataFrame'>
Index: 460996 entries, 1 to 986
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Period        460996 non-null  datetime64[ns]
 1   ReporterISO   460996 non-null  object        
 2   ReporterDesc  460996 non-null  object        
 3   PartnerISO    460996 non-null  object        
 4   PartnerDesc   460996 non-null  object        
 5   PrimaryValue  460996 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 24.6+ MB


In [24]:
### Can be deleted -- Used to explore the last appearance of country

# Find the last periods for each country in 'ReporterDesc'
last_appearance_reporter = (
    monthly_trade_data.groupby('ReporterDesc')['Period']
    .max()
    .reset_index()
    .rename(columns={'Period': 'LastPeriodReporter'})
)

# Find the last periods for each country in 'PartnerDesc'
last_appearance_partner = (
    monthly_trade_data.groupby('PartnerDesc')['Period']
    .max()
    .reset_index()
    .rename(columns={'Period': 'LastPeriodPartner'})
)

# Combine both Reporter and Partner appearances into a single DataFrame
last_appearance = pd.merge(
    last_appearance_reporter,
    last_appearance_partner,
    left_on='ReporterDesc',
    right_on='PartnerDesc',
    how='outer'
)

# Get the number of unique periods for LastPeriodReporter and LastPeriodPartner
num_unique_periods_reporter = last_appearance['LastPeriodReporter'].nunique()
num_unique_periods_partner = last_appearance['LastPeriodPartner'].nunique()

# Get the count of each unique period
period_counts_reporter = last_appearance['LastPeriodReporter'].value_counts()
period_counts_partner = last_appearance['LastPeriodPartner'].value_counts()

# Display the count of each unique period
print("Last Period Reporter Counts:")
print(period_counts_reporter.head())
print("\nLast Period Partner Counts:")
print(period_counts_partner.head())


Last Period Reporter Counts:
LastPeriodReporter
2023-08-01    77
2022-12-01    10
2023-07-01     6
2019-12-01     4
2021-12-01     4
Name: count, dtype: int64

Last Period Partner Counts:
LastPeriodPartner
2023-08-01    173
2023-07-01      8
2023-05-01      3
2023-06-01      2
2023-04-01      2
Name: count, dtype: int64


Get the country list for all countries appeared in monthly_trade_data

In [25]:

def get_country_list(df):
    """
    Extracts a unique list of countries from the DataFrame.

    Parameters:
    df (DataFrame): The input DataFrame with trade data.

    Returns:
    DataFrame: A DataFrame containing unique countries with their ISO codes and names.
    """
    # Select only the necessary columns
    df = df[['ReporterISO', 'ReporterDesc', 'PartnerISO', 'PartnerDesc']]

    # Create a DataFrame for Reporter countries
    reporter_countries = df[['ReporterISO', 'ReporterDesc']].rename(columns={'ReporterISO': 'ISO', 'ReporterDesc': 'Country'})
    
    # Create a DataFrame for Partner countries
    partner_countries = df[['PartnerISO', 'PartnerDesc']].rename(columns={'PartnerISO': 'ISO', 'PartnerDesc': 'Country'})
    
    # Concatenate both DataFrames and drop duplicates
    countries = pd.concat([reporter_countries, partner_countries]).drop_duplicates().reset_index(drop=True)
    
    return countries

# Apply the function to the trade data DataFrame
countries = get_country_list(monthly_trade_data)

# Read GDP data of countries
country_gdp = pd.read_excel('../data/raw/P_Data_Extract_From_World_Development_Indicators_GDP.xlsx', sheet_name='Data', index_col=None)

# Merge the country list with the GDP data
country_info = pd.merge(country_gdp, countries[['ISO']], how='right', on='ISO')

# Display the first few rows of the country_info DataFrame
print(country_info.head())

# Display a concise summary of the country_info DataFrame
print(country_info.info())

# Save the merged data to a CSV file
country_info.to_csv('../data/processed/countries.csv', index=False, header=True)

# Optional: Save to Excel file if needed
# country_info.to_excel('../data/processed/countries.xlsx', index=False)


         Name  ISO             GDP_2022              GDP_2023
0     Algeria  DZA  225560256621.757202   239899491127.742371
1      Angola  AGO  104399746853.401413    84722957642.375656
2  Azerbaijan  AZE   78807470588.235306    72356176470.588242
3   Argentina  ARG  631133384439.944458   640591410663.883423
4   Australia  AUS  1692956646855.70166  1723827215334.706299
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      192 non-null    object
 1   ISO       193 non-null    object
 2   GDP_2022  192 non-null    object
 3   GDP_2023  192 non-null    object
dtypes: object(4)
memory usage: 6.2+ KB
None


In [26]:

# Select only the necessary columns from the DataFrame
monthly_trade_data = monthly_trade_data[['Period', 'ReporterISO', 'PartnerISO', 'PrimaryValue']]

# Save the DataFrame to a CSV file
monthly_trade_data.to_csv('../data/processed/monthly_trade_data.csv', index=False, header=True)


### Get feature data

TPU -- Trade policy uncertainty index https://www.policyuncertainty.com/trade_cimpr.html

In [27]:

# Read data from Excel file
tpu = pd.read_excel('../data/raw/tpu_web_latest.xlsx', sheet_name='TPU_MONTHLY', index_col=None)

# Select only the necessary columns
tpu = tpu[['DATE', 'TPU']]

# Rename 'DATE' to 'Period'
tpu = tpu.rename(columns={'DATE': 'Period'})

# Convert 'Period' to datetime format
tpu['Period'] = pd.to_datetime(tpu['Period'], format='%Y-%m-%d')

# Filter rows where 'Period' is between 2010-01-01 and 2023-08-01
tpu = tpu[(tpu['Period'] >= '2010-01-01') & (tpu['Period'] <= '2023-08-01')]

# Display the first few rows of the DataFrame
print(tpu.head())

# Display a concise summary of the DataFrame
print(tpu.info())


        Period        TPU
600 2010-01-01  24.490808
601 2010-02-01  18.847337
602 2010-03-01  28.556673
603 2010-04-01  25.487728
604 2010-05-01  21.410693
<class 'pandas.core.frame.DataFrame'>
Index: 164 entries, 600 to 763
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Period  164 non-null    datetime64[ns]
 1   TPU     164 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.8 KB
None


GPR -- Geopolitical Risk Index https://www.matteoiacoviello.com/gpr.htm

In [28]:

# Read data from Excel file
gpr = pd.read_excel('../data/raw/data_gpr_export.xls', sheet_name='Sheet1', index_col=None)

# Rename 'month' column to 'Period'
gpr = gpr.rename(columns={'month': 'Period'})

# Convert 'Period' to datetime format
gpr['Period'] = pd.to_datetime(gpr['Period'], format='%YM%m')

# Filter the rows where 'Period' is between 2010-01-01 and 2023-08-01
gpr = gpr[(gpr['Period'] >= '2010-01-01') & (gpr['Period'] <= '2023-08-01')]

# Select only the 'Period' and 'GPR' columns
gpr = gpr[['Period', 'GPR']]

# Display the first few rows of the DataFrame
print(gpr.head())

# Display a concise summary of the DataFrame
print(gpr.info())


         Period        GPR
1320 2010-01-01  91.581024
1321 2010-02-01  80.725357
1322 2010-03-01  74.116943
1323 2010-04-01  88.761581
1324 2010-05-01  88.958710
<class 'pandas.core.frame.DataFrame'>
Index: 164 entries, 1320 to 1483
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Period  164 non-null    datetime64[ns]
 1   GPR     164 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.8 KB
None


GEM -- Global economic monitor index https://databank.worldbank.org/source/global-economic-monitor-(gem)#

In [29]:

# Read data from Excel file
gem = pd.read_excel('../data/raw/P_Data_Extract_From_Global_Economic_Monitor_(GEM)-2.xlsx', sheet_name='Data', index_col=None)

# Transpose the DataFrame and select the data starting from the fourth row
gem = gem.transpose().iloc[3:]

# Set the first row as the header
gem.columns = gem.iloc[0]
gem = gem[1:]

# Move the existing index to a regular column
gem.reset_index(inplace=True)

# Rename the column 'index' to 'Period'
gem = gem.rename(columns={'index': 'Period'})

# Extract the part before the blank space in the 'Period' column
gem['Period'] = gem['Period'].str.split().str[0]

# Convert 'Period' to datetime format
gem['Period'] = pd.to_datetime(gem['Period'], format='%YM%m')

# Filter rows where 'Period' is between 2010-01-01 and 2023-08-01
gem = gem[(gem['Period'] >= '2010-01-01') & (gem['Period'] <= '2023-08-01')]

# Replace '..' with NA
gem.replace('..', np.nan, inplace=True)

# Drop columns with all NaN values
gem = gem.dropna(axis=1, how='all')


In [30]:

# Read data
GEPU = pd.read_csv('../data/raw/GEPUCURRENT.csv', encoding='unicode_escape')

# Rename columns
GEPU = GEPU.rename(columns={'DATE': 'Period','GEPUCURRENT':'GEPU'})

# Keep only Period  between 2000M01 and 2023M09
GEPU['Period'] = pd.to_datetime(GEPU['Period'])
GEPU = GEPU[(GEPU['Period'] >= '2010-01-01') & (GEPU['Period'] <= '2023-08-01')]


In [31]:

# Merge 'gem' with 'GEPU' on the 'Period' column
gem = pd.merge(gem, GEPU, on='Period')

# Keep only the specified columns
gem = gem[['Period', 'CPTOTSAXN', 'DPANUSSPB', 'DXGSRMRCHSACD', 'DMGSRMRCHSACD', 'IPTOTSAKD', 'IMPCOV', 'NEER', 'REER', 'RETSALESSA', 'TOTRESV', 'UNEMPSA_', 'GEPU']]

# Display the cleaned DataFrame
print(gem.head())
print(gem.info())


      Period   CPTOTSAXN   DPANUSSPB  DXGSRMRCHSACD  DMGSRMRCHSACD  \
0 2010-01-01  100.000000  273.668678   1.094645e+06   1.802235e+06   
1 2010-02-01  100.147360  275.136917   1.108186e+06   1.853262e+06   
2 2010-03-01  100.297261  273.147971   1.128765e+06   1.819304e+06   
3 2010-04-01  100.477298  271.460411   1.132969e+06   1.811889e+06   
4 2010-05-01  100.611906  274.960784   1.118128e+06   1.774453e+06   

      IPTOTSAKD    IMPCOV        NEER        REER  RETSALESSA       TOTRESV  \
0  1.279704e+12  4.880311   99.710300   99.882075  100.970001  8.795469e+06   
1  1.283712e+12  4.751241  100.058497  100.144656  101.121526  8.805293e+06   
2  1.300778e+12  4.882158   99.910024   99.868581  102.100177  8.882128e+06   
3  1.309634e+12  4.978065   99.826979   99.736114   95.659314  9.019701e+06   
4  1.319176e+12  5.057788  100.486460  100.390645   96.108964  8.974806e+06   

   UNEMPSA_        GEPU  
0  8.266556  113.055131  
1  8.206897  111.376776  
2  8.188579  107.563775  


Construct feature matrix

In [32]:
# Merge tpu, gpr, gem into the features data
merged_data = pd.merge(tpu, gpr, on='Period')
features = pd.merge(merged_data, gem, on='Period')

# Save the merged DataFrame to a CSV file
features.to_csv('../data/processed/features.csv', index=False)

In [33]:
# Display a concise summary of the DataFrame
print(features.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Period         164 non-null    datetime64[ns]
 1   TPU            164 non-null    float64       
 2   GPR            164 non-null    float64       
 3   CPTOTSAXN      164 non-null    float64       
 4   DPANUSSPB      145 non-null    float64       
 5   DXGSRMRCHSACD  164 non-null    float64       
 6   DMGSRMRCHSACD  164 non-null    float64       
 7   IPTOTSAKD      164 non-null    float64       
 8   IMPCOV         164 non-null    float64       
 9   NEER           164 non-null    float64       
 10  REER           164 non-null    float64       
 11  RETSALESSA     164 non-null    float64       
 12  TOTRESV        164 non-null    float64       
 13  UNEMPSA_       164 non-null    float64       
 14  GEPU           164 non-null    float64       
dtypes: datetime64[ns](1), f

### Stock Price: Semiconductor ETF 
https://finance.yahoo.com/quote/SOXX/

In [34]:

# Read the ETF price data from CSV
etf_price = pd.read_csv('../data/raw/SOXX.csv')

# Rename columns and select needed columns
etf_price['Period'] = pd.to_datetime(etf_price['Date'])
etf_price['etf_price'] = etf_price['Adj Close']
etf_price = etf_price[['Period', 'etf_price']]

# Filter the 'Period' column to keep data between 2010-01-01 and 2023-08-01
etf_price = etf_price[(etf_price['Period'] >= '2010-01-01') & (etf_price['Period'] <= '2023-08-01')]

# Display the first few rows and a concise summary of the DataFrame
print(etf_price.head())
print(etf_price.info())

# Save the processed data to a CSV file
etf_price.to_csv('../data/processed/etf_price.csv', index=False, header=True)


        Period  etf_price
101 2010-01-01  12.225553
102 2010-02-01  13.201204
103 2010-03-01  14.057395
104 2010-04-01  14.341842
105 2010-05-01  13.403166
<class 'pandas.core.frame.DataFrame'>
Index: 164 entries, 101 to 264
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Period     164 non-null    datetime64[ns]
 1   etf_price  164 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.8 KB
None
